In [1]:
# Generic OpenAI-compatible endpoint using ChatOpenAI
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage,SystemMessage
import os
default_model_name = os.environ["OPENAI_MODEL"]
llm = ChatOpenAI(model=default_model_name)


In [2]:
messages = [HumanMessage("What is the capital of France?")]
llm.invoke(messages) 

AIMessage(content='The capital of France is **Paris**.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 0, 'prompt_tokens': 0, 'total_tokens': 0, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'zai', 'system_fingerprint': None, 'id': 'chatcmpl-1765803455541', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b2216-58ea-7111-baef-e417a643a7c3-0', usage_metadata={'input_tokens': 0, 'output_tokens': 0, 'total_tokens': 0, 'input_token_details': {}, 'output_token_details': {}})

In [3]:
system_msg = SystemMessage(
    '''You are a helpful assistant that responds to questions with three
        exclamation marks.'''
)
messages = [system_msg,HumanMessage("What is the capital of France?")]
llm.invoke(messages) 

AIMessage(content='The capital of France is Paris!!!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 37, 'total_tokens': 44, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen3-coder-plus', 'system_fingerprint': None, 'id': 'chatcmpl-620538dc-967a-44d9-b430-4d35ca5c918e', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b21ce-4de9-7e30-b7e8-951456a0b804-0', usage_metadata={'input_tokens': 37, 'output_tokens': 7, 'total_tokens': 44, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [6]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template("""Answer the question based on the
    context below. If the question cannot be answered using the information
    provided, answer with "I don't know".

Context: {context}

Question: {question}

Answer: """)

In [3]:
from pydantic import BaseModel
class AnswerWithJustification(BaseModel):
    '''An answer to the user's question along with justification for the answer.'''
    answer: str
    '''The answer to the user's question'''
    justification: str
    '''Justification for the answer'''

structured_llm = llm.with_structured_output(AnswerWithJustification)

structured_llm.invoke("""What weighs more, a pound of bricks or a pound of feathers""")

ValidationError: 1 validation error for AnswerWithJustification
  Invalid JSON: expected value at line 1 column 1 [type=json_invalid, input_value='This is a classic riddle... than a pound of bricks', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/json_invalid